In [5]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
from sportmonks import *
import re

#    the token of the dataset api
token = 'SLEzPmfrJIBg6ReqmSLArV9C33CYmxXWBApCxfzIlVDQOFdC7i0Iq5WhZCO3'
#    connect to the api
init(token)

# Create a trainer that uses this config
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('training_dataset.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

'''
message = 'can you provide the information about the commentaries'
parse_data = interpreter.parse(message)
print(parse_data)
intent = parse_data['intent']['name']
entities = parse_data['entities']
print(intent)
'''


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/ljq/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ljq/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ljq/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ljq/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'pre

"\nmessage = 'can you provide the information about the commentaries'\nparse_data = interpreter.parse(message)\nprint(parse_data)\nintent = parse_data['intent']['name']\nentities = parse_data['entities']\nprint(intent)\n"

In [ ]:
import itchat
import re
from sportmonks import *
import random
import urllib 
import requests 


# regular pattarn
pattern = re.compile(r'[0-9]{2}')
rule = r'[0-9]{4}-[0-9]{2}-[0-9]{2}'    #    date pattern



prior_fixture = {}     #    preserve the certain fixtures in the prior step
prior_fixtures = {}    #    preserve the fixtures of a certain period in the prior step
STATE = 0              #    0:welcome 1:fixtures_search 2:fixture_info 3:certain_info    state_auto_machine

thanks_reply = ['you are welcome','my pleasure']
welcome_reply = ['hello, I am your soccer fixtures searching bot Sofix. what can I help you? :)',
                 'hey, I am your soccer fixtures searching assistant Sofix. are you going to search some special informations? :)']



#    find all the matches in certain period 
def findFixtures(entities,message):
    
    entity = entities[0]
    reply = ''
    
    global prior_fixtures
   
    #    find the fixtures through two dates
    if entity['entity'] == 'between':
        match = re.findall(rule, message)       #    extract two dates
        table = fixtures(match[0],match[1])     #    the datas of the fixtures
        prior_fixtures = table                  #    preserve the datas of fixtures for following reply
        
        # no fixture in that period
        if table == None:
            reply = 'there is no soccor games held between '+ match[0] + ' and ' + match[1] + '.'
        else:
            # get the information from the json dataset
            for f in table:
                local_team = f['localteam_id']
                visit_team = f['visitorteam_id']
                response = team(local_team)['name'] + ' vs ' + team(visit_team)['name'] + '\n in ' + f['time']['starting_at']['date']
                reply = reply + response + '\n\n'
    # find the fixtures through one date       
    elif entity['entity'] == 'in a day':   
        match = re.findall(rule, message)       #    extract the date 
        table = fixtures(match[0])              #    the datas of the fixtures
        prior_fixtures = table                  #    preserve the datas of fixtures for following reply
        
        # no fixture in that period
        if table == None:
            reply = 'there is no soccor games held in '+ match[0] + '.'
        else:
            # get the information from the json dataset
            for f in table:
                print(f)
                local_team = f['localteam_id']
                visit_team = f['visitorteam_id']
                response = team(local_team)['name'] + ' vs ' + team(visit_team)['name'] + '\n in ' + f['time']['starting_at']['date']
                reply = reply + response + '\n\n'
        
    return reply



#    deal with the self-introduction from the user
def selfintroductionReply(msg):
    
    message = msg['Text']    #    get the text message                    
    
    #    the regular pattern of names
    name_pattern = re.compile('[A-Z]{1}[a-z]*')
    #    get the matching words in the string
    name_words = name_pattern.findall(message)
    name_words = name_words[1:]
    if len(name_words) > 0:
        #    return the name if the keywords are present
        name = ' '.join(name_words)
    else: 
        name = name_words
    
    #    send the reply
    reply = 'hello, ' + name + '. you can call me Sofix. what can I help you? :)'
    FromUserName = msg['FromUserName'];
    itchat.send(reply, toUserName=FromUserName)

    
    

#    analyse the message in the welcome state
def analysis_msg1(msg):
    
    reply = ''
    global STATE
    
    #    parse the message, get the intent and entities in that setence
    message = msg['Text']
    parse_data = interpreter.parse(message)
    intent = parse_data['intent']['name']
    entities = parse_data['entities']
    
    #    intent is hello
    if intent == 'hello':
        reply = random.choice(welcome_reply)                #    randomly choose a welcome reply
        FromUserName = msg['FromUserName'];                 #    get the id of users
        itchat.send(reply, toUserName=FromUserName)         #    send reply
    
    #    intent is finding the fixtures in certain period
    elif intent == 'findFixtures':
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        
        #    get the reply through the findFixture function 
        reply = findFixtures(entities,message)
        itchat.send(reply, toUserName=FromUserName)
        itchat.send('would you like to adopt another specific period for searching?', toUserName=FromUserName)
        itchat.send('or do you want more details about certain games above?', toUserName=FromUserName)
        
        #    change the state to fixtures state, which can further search for the certain fixture
        STATE = 1
        
    #    intent is self-introduction
    elif intent == 'selfIntroduction':
        reply = selfintroductionReply(msg)
    elif intent == 'thanks':
        FromUserName = msg['FromUserName'];
        itchat.send(random.choice(thanks_reply), toUserName=FromUserName)
        
        
        
#    find certain fixture through the name of the local team and the name of the visitor team
def findFixture(msg):
    
    #    get the name of the local team and the name of the visitor team
    words = msg['Text'].split(' ')
    game1 = ''
    game2 = ''
    game_info = {}
    for i in range(0,len(words)):
        if words[i] == 'vs':
            game1 = words[i-1]    #    the name of the local team
            game2 = words[i+1]    #    the name of the visitor team

    
    #    find the certain fixture in fixtures 
    global prior_fixtures
    for f in prior_fixtures:     #    get the data of the fixtures preserved in the prior messages
        if team(f['localteam_id'])['name'] == game1 and team(f['visitorteam_id'])['name'] == game2:
            game_info = f
            break
    
    global prior_fixture
    prior_fixture = game_info    #    preserve the information of the fixture for following replys
    
    #    reply for the requirement for certain fixture 
    FromUserName = msg['FromUserName'];
    reply = 'local team : ' + game1 + '\nvisitor team : ' + game2 + '\nscore : ' + str(game_info['scores']['localteam_score']) + '-' + str(game_info['scores']['visitorteam_score'])
    itchat.send(reply, toUserName=FromUserName)
    
    
    
        
#    analyse the message in the fixtures state  
def analysis_msg2(msg):
    
    global STATE
    reply = ''
    
    #    parse the message, get the intent and entities in that setence
    message = msg['Text']
    parse_data = interpreter.parse(message)
    intent = parse_data['intent']['name']
    entities = parse_data['entities']
    #print(intent)
    
    #    intent is findFixture, go to the function findFixture
    if intent == 'findFixture':
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        findFixture(msg)
        STATE = 2    #    change the state to the fixture state
    #    intent is findFixtures, go to the function findFixtures, the state unchanges
    elif intent == 'findFixtures':  
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        reply = findFixtures(entities,message)
        itchat.send(reply, toUserName=FromUserName)
        itchat.send('would you like to adopt another period for searching?', toUserName=FromUserName)
        itchat.send('or do you want more details about certain games above?', toUserName=FromUserName)
    elif intent == 'thanks':
        FromUserName = msg['FromUserName'];
        itchat.send(random.choice(thanks_reply), toUserName=FromUserName)
     
        

#    find certain information in a fixture such as weather, venue, coaches and so on
def findCertainInfo(msg):
    
    #    parse the message, get the FromUserName and entities in that setence
    message = msg['Text']
    parse_data = interpreter.parse(message)
    entity = parse_data['entities'][0]['entity']
    FromUserName = msg['FromUserName'];
    #print(entity)
    
    #    entity judgement 
    if entity == 'formation':
        findFormation(FromUserName)
    elif entity == 'weather':
        findWeather(FromUserName)
    elif entity == 'result':
        findResult(FromUserName)
    elif entity == 'localteam':
        findLocalteam(FromUserName)
    elif entity == 'coach':
        findCoach(FromUserName)
    elif entity == 'starting_time':
        findStartingtime(FromUserName)
    elif entity == 'visitorteam':
        findVisitorteam(FromUserName)
    elif entity == 'venue':
        findVenue(FromUserName)
    elif entity == 'commentaries':
        findCommentaries(FromUserName)
    elif entity == 'all teams':
        findAllteams(FromUserName)

        
        
#    find formations in certain fixture
def findFormation(FromUserName):
    
    print('in')
    reply = 'localteam_formation : ' + prior_fixture['formations']['localteam_formation'] + '\nvisitorteam_formation : ' + prior_fixture['formations']['visitorteam_formation'] 
    itchat.send(reply, toUserName=FromUserName)

    
    
#    find weather in certain fixture
def findWeather(FromUserName):
    
    #print('in')
    #    send the text message
    reply = 'weather : ' + prior_fixture['weather_report']['type'] + '\ntemperature : ' + str(prior_fixture['weather_report']['temperature']['temp']) + ' ' + prior_fixture['weather_report']['temperature']['unit']
    reply = reply + '\npressure : ' + str(prior_fixture['weather_report']['pressure'])
    reply = reply + '\nhumidity : ' + prior_fixture['weather_report']['humidity']
    reply = reply + '\nwind : ' + prior_fixture['weather_report']['wind']['speed']
    itchat.send(reply, toUserName=FromUserName)
    
    #    set the config of request
    opener = urllib.request.build_opener()
    #    add header in case of being in 403
    opener.addheaders = [('User-agent', 'Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10')]
    urllib.request.install_opener(opener)
    url = prior_fixture['weather_report']['icon']
    #    get image from url
    urllib.request.urlretrieve(url, 'weather_icon.png')
    #    send image to user
    itchat.send_image('weather_icon.png',toUserName=FromUserName)
    
    
    
#    find result in certain fixture
def findResult(FromUserName):
    
    reply = 'whole score : ' + str(prior_fixture['scores']['localteam_score']) + '-' + str(prior_fixture['scores']['visitorteam_score'])
    reply = reply + '\nht score : ' + prior_fixture['scores']['ht_score'] + '\nft score : ' + prior_fixture['scores']['ft_score'] 
    itchat.send(reply, toUserName=FromUserName) 

    

#    find coaches in certain fixture                                                                                                               
def findCoach(FromUserName):
    
    #    add header in case of being in 403
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-agent', 'Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10')]
    urllib.request.install_opener(opener)
    
    #    send the text message of the coach from the local team
    reply = 'the coach of the local team' + '\nname : ' + coach(prior_fixture['coaches']['localteam_coach_id'])['common_name'] + '\nnationality : ' +  coach(prior_fixture['coaches']['localteam_coach_id'])['nationality'] + '\nbirthdate : ' + coach(prior_fixture['coaches']['localteam_coach_id'])['birthdate']     
    itchat.send(reply, toUserName=FromUserName) 
    url = coach(prior_fixture['coaches']['localteam_coach_id'])['image_path']
    #    send image to user
    urllib.request.urlretrieve(url, 'coach_image.png')
    itchat.send_image('coach_image.png',toUserName=FromUserName)
    
    #    send the text message of the coach from the visitor team
    reply = 'the coach of the visitor team' + '\nname : ' + coach(prior_fixture['coaches']['visitorteam_coach_id'])['common_name'] + '\nnationality : ' +  coach(prior_fixture['coaches']['visitorteam_coach_id'])['nationality'] + '\nbirthdate : ' + coach(prior_fixture['coaches']['visitorteam_coach_id'])['birthdate'] 
    itchat.send(reply, toUserName=FromUserName)
    url = coach(prior_fixture['coaches']['visitorteam_coach_id'])['image_path']
    #    send image to user
    urllib.request.urlretrieve(url, 'coach_image.png')
    itchat.send_image('coach_image.png',toUserName=FromUserName)
                
        
        
#    find starting time in certain fixture   
def findStartingtime(FromUserName):
    
    reply = 'the starting time : ' + prior_fixture['time']['starting_at']['date_time']
    itchat.send(reply, toUserName=FromUserName) 

    
    
#    find details about the local team in certain fixture 
def findLocalteam(FromUserName):
    
    reply = 'the information of the local team' 
    itchat.send(reply, toUserName=FromUserName)    
    
    reply = 'name : ' + team(prior_fixture['localteam_id'])['name']
    reply = reply + '\ncountry : ' + country(team(prior_fixture['localteam_id'])['country_id'])['name']
    reply = reply + '\nfounded : ' + str(team(prior_fixture['localteam_id'])['founded'])
    reply = reply + '\nvenue : ' + venue(team(prior_fixture['localteam_id'])['venue_id'])['name']
    itchat.send(reply, toUserName=FromUserName) 
                                                                                           

        
#    find details about the visitor team in certain fixture
def findVisitorteam(FromUserName):
    
    reply = 'the information of the visitor team' 
    itchat.send(reply, toUserName=FromUserName)    
    
    reply = 'name : ' + team(prior_fixture['visitorteam_id'])['name']
    reply = reply + '\ncountry : ' + country(team(prior_fixture['visitorteam_id'])['country_id'])['name']
    reply = reply + '\nfounded : ' + str(team(prior_fixture['visitorteam_id'])['founded'])
    reply = reply + '\nvenue : ' + venue(team(prior_fixture['visitorteam_id'])['venue_id'])['name']
    itchat.send(reply, toUserName=FromUserName) 
    
    
                                                                                           
#    find details about the venue in certain fixture
def findVenue(FromUserName):
    reply = 'the information of the venue'
    itchat.send(reply, toUserName=FromUserName)                                                                                        
                                                                                        
    reply = 'name : ' + venue(prior_fixture['venue_id'])['name']                                                                                       
    reply = reply + '\nsurface : ' + venue(prior_fixture['venue_id'])['surface']
    reply = reply + '\naddress : ' + venue(prior_fixture['venue_id'])['address']                                                                                       
    reply = reply + '\ncity : ' + venue(prior_fixture['venue_id'])['city']  
    reply = reply + '\ncapacity : ' + str(venue(prior_fixture['venue_id'])['capacity'])      
    itchat.send(reply, toUserName=FromUserName)                                                                                        

    
    
#    find commentaries in certain fixture                                                                          
def findCommentaries(FromUserName):
    
    global STATE
    reply = ''
    #    no commentaries
    if prior_fixture['commentaries'] == False:
        itchat.send('no commentary for that game', toUserName=FromUserName)
    else:
        #    get commentaries from json dataset
        commentaries_ = commentaries(prior_fixture['id'])
        STATE = 4    #    change the state to commentaries spliting state                                                                 
        for c in commentaries_:
            if c['extra_minute'] != None:
                time = str(c['minute'] + c['extra_minute'])    #    calculate the time
            else:
                time = str(c['minute'])
            reply = reply + '\n\n' + time + '\'　: ' + c['comment']    #    send the commentary
        itchat.send(reply, toUserName=FromUserName)                                                                                  

            
            
#    find all teams in that season              
def findAllteams(FromUserName):
    
    fixtures = teams(prior_fixture['season_id'])                                                                                           
    reply = ''                                                                                       
    for f in fixtures:                                                                                       
        reply = reply + f['name'] + '\n'                                                                                   
                                                                                           
    itchat.send(reply, toUserName=FromUserName)                                                                                       
                                                                                           

        
        
#    analyse the message in the fixture state                                                                                                                
def analysis_msg3(msg):
    
    global STATE
    reply = ''
    
    #    parse the message, get the intent and entities in that setence
    message = msg['Text']
    parse_data = interpreter.parse(message)
    intent = parse_data['intent']['name']
    entities = parse_data['entities']
    print(intent)
    
    #    intent is findFixture, go to findFixture function, state unchanged
    if intent == 'findFixture':
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        findFixture(msg)
    #    intent is findFixtures, go to findFixtures function, state is changed to fixtures state 
    elif intent == 'findFixtures':
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        reply = findFixtures(entities,message)
        
        itchat.send(reply, toUserName=FromUserName)
        itchat.send('would you like to adopt another specific period for searching?', toUserName=FromUserName)
        itchat.send('or do you want more details about certain games above?', toUserName=FromUserName)
        
        STATE = 1
    #    intent is findCertainInfo, go to findCertainInfo function, state is changed to details state 
    elif intent == 'findCertainInfo':
        STATE = 3
        findCertainInfo(msg)
    elif intent == 'thanks':
        FromUserName = msg['FromUserName'];
        itchat.send(random.choice(thanks_reply), toUserName=FromUserName)
    

    
     
#    analyse the message in the details state    
def analysis_msg4(msg):
    
    global STATE
    reply = ''
    
    #    parse the message, get the intent and entities in that setence
    message = msg['Text']
    parse_data = interpreter.parse(message)
    intent = parse_data['intent']['name']
    entities = parse_data['entities']
    print(intent)
    
    #    intent is findFixture, go to findFixture function, state is changed to fixture state
    if intent == 'findFixture':
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        findFixture(msg)
        STATE = 2
    #    intent is findFixtures, go to findFixtures function, state is changed to fixturess state
    elif intent == 'findFixtures':
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        reply = findFixtures(entities,message)
        
        itchat.send(reply, toUserName=FromUserName)
        itchat.send('would you like to adopt another specific period for searching?', toUserName=FromUserName)
        itchat.send('or do you want more details about certain games above?', toUserName=FromUserName)
     
        STATE = 1
    #    intent is findCertainInfo, go to findCertainInfo function, state unchanged    
    elif intent == 'findCertainInfo':
        findCertainInfo(msg)
    elif intent == 'thanks':
        FromUserName = msg['FromUserName'];
        itchat.send(random.choice(thanks_reply), toUserName=FromUserName)

        
        
#    find certain comments, split the unneccesary comments
def findCertaincomments(msg):
    
    #    parse the message, get the entities in that setence
    message = msg['Text']
    parse_data = interpreter.parse(message)
    entity = parse_data['entities'][0]['entity']
    FromUserName = msg['FromUserName'];
    print(entity)

    reply = ''
    #    entity is goal, get all the commentaries about the goal
    if entity == 'goal':
        commentaries_ = commentaries(prior_fixture['id'])   
        for c in commentaries_:
            if c['goal'] == True:
                if c['extra_minute'] != None:
                    time = str(c['minute'] + c['extra_minute'])   
                else:
                    time = str(c['minute'])
                reply = reply + '\n\n' + time + '\'　: ' + c['comment']
        itchat.send(reply, toUserName=FromUserName)        
    #    entity is important, get all the important commentaries 
    
    elif entity == 'important':
        commentaries_ = commentaries(prior_fixture['id'])   
        for c in commentaries_:
            if c['important'] == True:
                if c['extra_minute'] != None:
                    time = str(c['minute'] + c['extra_minute'])   
                else:
                    time = str(c['minute'])
                reply = reply + '\n\n' + time + '\'　: ' + c['comment']
        itchat.send(reply, toUserName=FromUserName)   
    #    entity is visitorteam, get all the commentaries about the visitor team 
    
    elif entity == 'visitorteam':
        #    get the name of the visitor team
        team_name = team(prior_fixture['visitorteam_id'])['name']
        commentaries_ = commentaries(prior_fixture['id'])   
        for c in commentaries_:
            #    if comment is marked with the name of visitor team, send reply
            if c['comment'].find(team_name) != -1:
                if c['extra_minute'] != None:
                    time = str(c['minute'] + c['extra_minute'])   
                else:
                    time = str(c['minute'])
                reply = reply + '\n\n' + time + '\'　: ' + c['comment']
        itchat.send(reply, toUserName=FromUserName) 
                
    #    entity is visitorteam, get all the commentaries about the local team 
    elif entity == 'localteam':
        #    get the name of the local team
        team_name = team(prior_fixture['localteam_id'])['name']
        print(team_name)
        commentaries_ = commentaries(prior_fixture['id'])   
        for c in commentaries_:
            #    if comment is marked with the name of local team, send reply
            print(c['comment'].find(team_name))
            if c['comment'].find(team_name) != -1:
                if c['extra_minute'] != None:
                    time = str(c['minute'] + c['extra_minute'])   
                else:
                    time = str(c['minute'])
                reply = reply + '\n\n' + time + '\'　: ' + c['comment']
                print(c['comment'])
        itchat.send(reply, toUserName=FromUserName) 
    
      
        
#    analyse the message in the commentaries spliting state      
def analysis_msg5(msg):
    
    global STATE
    reply = ''
    
    #    parse the message, get the intent and entities in that setence
    message = msg['Text']
    parse_data = interpreter.parse(message)
    intent = parse_data['intent']['name']
    entities = parse_data['entities']
    print(intent)
    
    #    intent is findFixture, go to findFixture function, state is changed to fixture state
    if intent == 'findFixture':
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        findFixture(msg)
        STATE = 2
    #    intent is findFixtures, go to findFixtures function, state is changed to fixturess state
    elif intent == 'findFixtures':
        FromUserName = msg['FromUserName'];
        itchat.send('wait for a moment.....', toUserName=FromUserName)
        reply = findFixtures(entities,message)
        
        itchat.send(reply, toUserName=FromUserName)
        itchat.send('would you like to adopt another specific period for searching?', toUserName=FromUserName)
        itchat.send('or do you want more details about certain games above?', toUserName=FromUserName)
        
        STATE = 1
    #    intent is findCertainInfo, go to findCertainInfo function, state is changed to details state
    elif intent == 'findCertainInfo':    
        STATE = 3
        findCertainInfo(msg)  
    #    intent is findCommentaries, go to findCertaincomments function, state unchanged   
    elif intent == 'findCommentaries':
        print('in')
        findCertaincomments(msg)
    elif intent == 'thanks':
        FromUserName = msg['FromUserName'];
        itchat.send(random.choice(thanks_reply), toUserName=FromUserName)
        
        
    
    
    
@itchat.msg_register(itchat.content.TEXT)
def text_reply(msg):
    
    global STATE
    
    #print(STATE)
    # state judgement 
    if(STATE == 0):    
        analysis_msg1(msg)         #   welcome state
    elif(STATE == 1):
        analysis_msg2(msg)         #   fixtures state
    elif(STATE == 2):
        analysis_msg3(msg)         #   fixture state
    elif(STATE == 3):
        analysis_msg4(msg)         #   details state
    elif(STATE == 4):
        analysis_msg5(msg)         #   commentaries spliting state
   
        

#itchat.logout()                   #    if the log say it has logged in, you can log out through this code
itchat.auto_login()
itchat.run()


Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 扎扎
Start auto replying.
